In [ ]:
pip install langgraph langchain-community langchain-anthropic tavily-python typing-extensions langchain-openai ipywidgets


In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain.schema import SystemMessage, HumanMessage
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from IPython.display import Image, display, Markdown
from langchain_openai import ChatOpenAI
from typing import List, Dict, Optional, Any
from typing_extensions import TypedDict
from datetime import datetime, timedelta
import tkinter as tk
from tkinter import messagebox
import json
import os

#os.environ['ANTHROPIC_API_KEY'] = 'cb701c5a3a19d19c357030be6a4aef57eb93074730a4ca1e69c72108ddfa962b'
os.environ['TAVILY_API_KEY'] = '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'

#Definizione dello stato
class BlogState(TypedDict):
    suggested_topics: List[Dict]
    draft: List[str]
    draft_titles: List[str]
    selected_topic: Optional[str]
    last_research: List[Dict]
    schedule_post: List[Dict]
    memory: Dict[str, Any]

# Inizializzazione strumenti
#llm = ChatAnthropic(model="claude-3-haiku-20240307")
llm = ChatOpenAI(
    model="mistralai/Mistral-7B-Instruct-v0.2",
    openai_api_base="https://api.together.xyz",
    api_key="cb701c5a3a19d19c357030be6a4aef57eb93074730a4ca1e69c72108ddfa962b",
    temperature=0.7
)
tavily_tool = TavilySearchResults(max_results=5)


#|-------------------------------------------------------------------------------------------|

def publication_calendar(state: BlogState, intervallo_giorni: int = 3):
    if not state.get("draft_titles"):
        display(Markdown("⚠️ Nessuna bozza salvata disponibile per pianificare il calendario."))
        return state

    start_date = datetime.now()
    calendario = []

    for i, titolo in enumerate(state["draft_titles"]):
        data_pubblicazione = (start_date + timedelta(days=i * intervallo_giorni)).strftime("%d-%m-%Y")
        calendario.append({
            "date": data_pubblicazione,
            "title": titolo,
            "status": "scheduled"
        })

    state["schedule_post"] = calendario

    display(Markdown("📆 Calendario di pubblicazione dei post"))
    for entry in calendario:
        display(Markdown(f"- `{entry['date']}` → **{entry['title']}**"))

    return state

#|-------------------------------------------------------------------------------------------|

def suggest_topics(state: BlogState, user_input: str):
    recent_topics = state.get("draft_titles", [])[-10:]

    prompt = f"""**Task**: Genera un'idea per un post blog di tipo **'how-to'** (guida pratica), partendo dal tema: '{user_input}'. 
        Il post deve spiegare **come eseguire un'attività specifica**, dettagliando il processo e includendo suggerimenti utili.
        Il post deve essere chiaro, utile e adatto a un pubblico che sta cercando di **imparare o migliorare una competenza**.

        **Requisiti per l'idea**:
        - Deve includere procedure passo-passo, tecniche, o soluzioni pratiche a problemi comuni.
        - Deve includere argomenti che sono facilmente comprensibili e che possono essere implementati dai lettori con le risorse disponibili.
        - Può suggerire strumenti, materiali o risorse che potrebbero essere utili durante il processo descritto.

        **Vincoli**:
        1. Evitare questi topic recenti: {recent_topics}. Non ripetere argomenti trattati recentemente.
        2. Per ogni suggerimento fornire:
        - un **Titolo accattivante** che stimoli l'interesse del lettore e descriva chiaramente il contenuto della guida.
        - il **Tipo** del post (sempre "how-to" in questo caso).
        - una **Motivazione** (anche più di una frase) che spiega perché il post è utile e come risolverà un problema o semplificherà una attività per il lettore.
        3. Gli argomenti devono essere pratici e applicabili. **Esempi di categorie potrebbero includere**:
        - Tecniche manuali (es. "Come fare la manutenzione di un attrezzo da giardino").
        - Software e strumenti (es. "Come ottimizzare le performance di un sito web").
        - Attività quotidiane o di vita pratica (es. "Come organizzare un armadio in modo efficiente").
        4. Il contenuto deve essere scritto in modo che il lettore possa facilmente seguire le istruzioni **senza conoscenze pregresse**, includendo eventualmente consigli o risorse utili.
        5. Ogni idea deve essere originale e non troppo simile ad altri post già suggeriti.

        **Lingua dell'output**: l'intero risultato deve essere scritto in **italiano**.

        **Formato dell' output**:
        ```json
        [
            {{
                "title": "Titolo del post",
                "type": "how-to",
                "justification": "Descrizione dettagliata dell'argomento e del processo, e perché è utile per i lettori."
            }},
            ...
        ]
        ```
    """

    #prova
    messages = [
        SystemMessage(content=(
            "Sei un esperto editor di contenuti per blog. Il tuo compito è creare idee originali per articoli di tipo 'how-to', "
            "pensati per un pubblico ampio e inesperto. Ogni proposta deve essere utile, facilmente applicabile e con struttura chiara."
        )),
        HumanMessage(content=prompt)
    ]
    response = llm.invoke(messages)
    #fine prova

    #response = llm.invoke(prompt)

    try:
        content = response.content
        if '```json' in content:
            content = content.split('```json')[1].split('```')[0]
        suggestions = json.loads(content)

        #Questo ti aiuta a garantire che ogni elemento della lista suggestions abbia una struttura coerente e valida prima di procedere con ulteriori operazioni.
        for topic in suggestions:
            if not all(key in topic for key in ['title', 'type', 'justification']):
                raise ValueError("Formato topic non valido")

        display(Markdown("🟢 Topic suggeriti:"))
        for i, t in enumerate(suggestions, 1):
            display(Markdown(f"{i}. {t['title']} ({t['type']}) - {t['justification']}"))

        selected_index = int(input("Scegli il numero del topic da usare: ")) - 1
        if 0 <= selected_index < len(suggestions):
            selected_topic = suggestions[selected_index]["title"]
            state["selected_topic"] = selected_topic
            display(Markdown(f"✅ Topic selezionato: {selected_topic}"))
        else:
            display(Markdown("⚠️ Selezione non valida"))
            state["selected_topic"] = None

        state["suggested_topics"] = suggestions
        return state

    except (json.JSONDecodeError, ValueError) as e:
        print(f"Errore nel parsing dei suggerimenti: {str(e)}")
        display(Markdown("🔴 Errore: problemi nella generazione dei suggerimenti"))
        return {
            "suggested_topics": [{
                "title": "Errore: suggerimenti non disponibili",
                "type": "error",
                "justification": "Prova a riformulare la richiesta"
            }],
        }

#|----------------------------------------------------------------------------------------------------------------------------|

def research_topic(state):
    if not state.get("selected_topic"):
        display(Markdown("⚠️ Nessun topic selezionato."))
        state["last_research"] = []
        return []

    topic = state["selected_topic"]
    query = f"{topic}. Guida pratica/tutorial dettagliato con istruzioni passo-passo"
    display(Markdown(f"Ricerca in corso: {query}"))

    try:
        raw_results = tavily_tool.invoke({
            "query": query,
            "search_depth": "advanced",
            "max_results": 5,
            "include_answer": False
        })
    except Exception:
        display(Markdown("🔴 Errore: Connessione al servizio di ricerca fallita."))
        state["last_research"] = []
        return []

    if not isinstance(raw_results, list) or not raw_results:
        display(Markdown("⚠️ Nessun risultato trovato."))
        state["last_research"] = []
        return []

    trusted_domains = [
        "wikipedia.org", "treccani.it", "internazionale.it", "ilpost.it",
        "corriere.it", "repubblica.it", "wired.it", "fanpage.it",
        "rai.it", "ansa.it", "ilfattoquotidiano.it", "altalex.com",
        "agi.it", "ilsole24ore.com", "quifinanza.it", "aranzulla.it",
        "nigiara.it", "ideegreen.it", "bricolage-faidate.it", "youtube.com"
    ]

    evaluated_results = []
    for result in raw_results:
        try:
            #es: "https://www.example.com/path" -> divide in due parti separando // -> recupera solo l'ultimo elemento (www.example.com/path) -> dopo divide ulteriormente e prende solo www.example.com
            domain = result.get("url", "").split("//")[-1].split("/")[0].lower()
            content = result.get("content", "")
            date = result.get("published_date", "")

            is_trusted = any(dom in domain for dom in trusted_domains)
            is_informative = len(content) > 300
            is_recent = any(year in date for year in ["2022", "2023", "2024", "2025"])

            score = sum([is_trusted, is_informative, is_recent])
            if score == 3:
                level = "alta"
            elif score == 2:
                level = "media"
            elif score == 1:
                level = "bassa"
            else:
                level = None

            if level:
                evaluated_results.append({
                    "title": result.get("title", "N/A"),
                    "url": result.get("url", ""),
                    "domain": domain,
                    "level": level,
                    "content": content,
                    "excerpt": content[:200] + "..."
                })
        except Exception:
            continue

    #il sort() viene utilizzato per ordinare una lista di evaluated_results in base al valore della chiave "level"
    #index restituisce l’indice dell’elemento nell’elenco, che sarà un valore numerico che permette di ordinare gli elementi in ordine crescente (prima “alta”, poi “media”, e infine “bassa”)
    evaluated_results.sort(key=lambda x: ["alta", "media", "bassa"].index(x["level"]))
    final_results = evaluated_results[:3]

    state["last_research"] = final_results

    if final_results:
        display(Markdown("✅ Fonti selezionate:"))
        for res in final_results:
            display(Markdown(f"- [{res['title']}]({res['url']})  \n`{res['domain']}` | Affidabilità: **{res['level']}**"))
    else:
        display(Markdown("⚠️ Nessun risultato valido"))

    return final_results

#|----------------------------------------------------------------------------------------------------------------------------|

def editor_tkinter(draft):
    updated_text = {"value": draft}

    def save_and_close():
        updated_text["value"] = text_area.get("1.0", tk.END).strip()
        messagebox.showinfo("Salvato", "La bozza è stata salvata.")
        editor.destroy()

    editor = tk.Tk()
    editor.title("Editor della Bozza")

    text_area = tk.Text(editor, wrap="word", width=100, height=25)
    text_area.insert("1.0", draft)
    text_area.pack(padx=10, pady=10)

    save_button = tk.Button(editor, text="Salva e Chiudi", command=save_and_close)
    save_button.pack(pady=(0, 10))

    editor.mainloop()
    return updated_text["value"]

def ask_input(prompt, valid_options):
    while True:
        user_input = input(prompt).strip().lower()
        if user_input in valid_options:
            return user_input
        else:
            display(Markdown(f"⚠️ Input non valido, scrivi uno tra: {', '.join(valid_options)}"))

def generate_post(state: BlogState):
    if "selected_topic" not in state or not state["selected_topic"]:
        display(Markdown("⚠️ Nessun topic selezionato. Scegli un topic dai suggerimenti prima di procedere."))
        return

    title = state["selected_topic"]
    display(Markdown(f"📝 Creazione bozza per il topic: {title}"))

    available_moods = ["professionale", "divertente", "motivazionale", "ironico"]
    print("Scegli uno stile di scrittura:")
    for idx, mood in enumerate(available_moods, 1):
        print(f"{idx}. {mood}")
    try:
        mood_index = int(input("Inserisci il numero dello stile desiderato: ")) - 1
        selected_mood = available_moods[mood_index] if 0 <= mood_index < len(available_moods) else "professionale"
    except Exception:
        selected_mood = "professionale"

    display(Markdown("🔎 Avvio ricerca per le fonti!"))
    research_topic(state)

    research = state.get("last_research", [])
    if not research:
        display(Markdown("⚠️ Nessuna fonte disponibile."))
        return

    resources_text = "\n".join([ f"- {res['title']} ({res['url']})" for res in research])

    if not resources_text.strip():
        display(Markdown("⚠️ Nessuna fonte utile (alta/media) disponibile per il post."))
        return

    draft_prompt = f"""**Obiettivo**: Scrivi una **bozza completa** in lingua italiana per un post di tipo **"how-to"** (guida pratica) sul seguente argomento: **Topic**: "{title}"
        ---
        **Risorse da utilizzare**: {resources_text}
        ---
        **Stile richiesto**: {selected_mood}
        ---
        ### Linee guida per la scrittura del post:

        1. **Tono e stile**:
        - Usa un linguaggio coerente con lo stile: {selected_mood}.
        - Il tono deve essere pratico, amichevole e chiaro, adatto anche a lettori senza conoscenze pregresse.
        - Coinvolgi il lettore fin dall’inizio con esempi concreti e applicazioni pratiche dove possibile.

        2. **Struttura del contenuto**:
        - **Titolo**: accattivante e coerente con il contenuto della guida.
        - **Introduzione**: spiega brevemente l’utilità della guida, a chi è rivolta e cosa imparerà il lettore.
        - **Materiali o strumenti necessari** (se rilevanti): elenca tutto ciò che serve per seguire i passaggi.
        - **Passaggi dettagliati**:
            - Presenta ogni passaggio in ordine logico.
            - Per ogni step: spiega *cosa fare*, *perché è importante* e *come farlo al meglio*, includendo eventuali suggerimenti o trucchi.
        - **Consigli pratici**: aggiungi suggerimenti per migliorare l’esperienza o ottenere risultati migliori.
        - **Errori comuni da evitare**: segnala eventuali difficoltà o scelte sbagliate frequenti.
        - **Risorse aggiuntive** (opzionali): link utili, strumenti online, riferimenti.
        - **Conclusione**: incoraggia il lettore a provare, condividere la propria esperienza, o fare domande.

        3. **Formato**:
        - Usa **elenchi numerati** per i passaggi.
        - Evidenzia in **grassetto** i concetti chiave o le parole importanti.
        - Mantieni paragrafi brevi e scorrevoli. Esegui una descrizione più dettagliata ove lo ritieni necessario.
        - Usa uno stile chiaro, diretto e positivo.

        ---

        ### Output atteso:
        Fornisci il contenuto del post come **testo semplice in lingua italiana**, ben formattato e pronto per essere rivisto o pubblicato. 
        **Non usare Markdown, codice, o formati strutturati come JSON.**
        """

    response = llm.invoke(draft_prompt)
    draft = response.content

    display(Markdown("✍️ Ecco una bozza del post:"))
    display(Markdown(draft))

    user_input = ask_input("Vuoi migliorare la bozza appena creata? (si/no): ", ["si", "no"])
    if user_input == "si":
        user_input_choice = ask_input("Vuoi aggiungere un feedback o modificare la bozza? Scegli (f/m): ", ["f", "m"])
        if user_input_choice == "m":
            updated_draft = editor_tkinter(draft)
            user_confirm = ask_input("Vuoi memorizzare la bozza modificata? (si/No): ", ["si", "no"])
            if user_confirm == "si":
                state["draft"].append(updated_draft)
                display(Markdown("✅ Bozza modificata salvata."))
            else:
                state["draft"].append(draft)
                display(Markdown("✅ Bozza originale salvata."))
        else:
            feedback = input("inserisci il tuo feedback: ").strip().lower()
            if not feedback:
                state["draft"].append(draft)
                display(Markdown("✅ Feedback non fornito, Bozza originale salvata."))
            else:
                display(Markdown("--------------------------------------------------"))
                display(Markdown(f"🟢 Hai scelto di revisione la bozza sulla base di questo feedback: {feedback}. Fase di revisione avviata.."))
                revision_prompt = f"""**Obiettivo**: Revisiona e migliora il seguente post di tipo **"how-to"** (guida pratica), tenendo conto del feedback fornito.
                    ---
                    ### **Bozza originale**: {draft}
                    ---
                    ### **Feedback ricevuto**:{feedback}
                    ---

                    ### **Istruzioni per la revisione**:
                    1. **Migliora il contenuto**:
                    - Approfondisci i punti segnalati nel feedback, fornendo spiegazioni più chiare o dettagliate e pratiche.
                    - Assicurati che i passaggi siano logici, ben spiegati e facilmente comprensibili anche per principianti.
                    - Integra esempi pratici, consigli utili o note aggiuntive dove opportuno.
                    - Mantieni la **coerenza con la bozza originale fornita sopra**: migliora senza stravolgere, a meno che il feedback lo richieda espressamente.

                    2. **Mantieni stile e tono**:
                    - Rispetta lo stile originale richiesto: **{selected_mood}**.
                    - Mantieni un tono pratico, coinvolgente e professionale.

                    3. **Controlla e migliora la struttura**:
                    - Verifica che il post rispetti la seguente struttura:
                        - **Titolo**: accattivante e coerente con il contenuto (es: usa verbi d’azione, benefici chiari).
                        - **Introduzione**: utile e orientata al lettore.
                        - **Elenco dei materiali/strumenti**: (se rilevante)elenca tutto ciò che serve per seguire i passaggi.
                        - **Passaggi numerati**: ben spiegati e ordinati logicamente.
                        - **Consigli pratici** e **errori comuni da evitare**.
                        - **Risorse aggiuntive** (opzionali).
                        - **Conclusione** chiara, con invito all’azione.

                    4. **Revisione linguistica e formattazione**:
                    - Correggi eventuali errori grammaticali, sintattici o stilistici.
                    - Usa **markdown** per strutturare il post:
                        - Titoli e sottotitoli per separare le sezioni.
                        - Elenchi puntati o numerati dove opportuno.
                        - Evidenzia in **grassetto** i concetti chiave.

                    ---

                    ### **Output atteso**:
                    - Restituisci il post revisionato in **Markdown**, pronto per la pubblicazione.
                    - Non aggiungere codice JSON, spiegazioni, disclaimer o testo fuori contesto.
                    - Non introdurre contenuti generici o non richiesti.
                    """
                
                response = llm.invoke(revision_prompt)
                update_draft = response.content
                display(Markdown("✍️ Bozza Revisionata:"))
                display(Markdown(update_draft))
                user_input_confirm = ask_input("Vuoi memorizzare la bozza revisionata? (si/no): ", ["si", "no"])
                if user_input_confirm == "si":
                    state["draft"].append(update_draft)
                    display(Markdown("✅ Bozza revisionata salvata."))
                else:
                    state["draft"].append(draft)
                    display(Markdown("✅ Bozza originale salvata."))

    else:
        state["draft"].append(draft)
        display(Markdown("✅ Bozza salvata senza modifiche."))
        state.setdefault("draft_titles", []).append(title)
        state["selected_topic"] = None
        display(Markdown("🔁 Seleziona ora un nuovo topic per generare un altro post."))

#|----------------------------------------------------------------------------------------------------------------------------|

def show_state(state: BlogState):
    display(Markdown("🟢 Stato attuale del BlogAgent"))
    draft = state.get("draft", None)
    if draft:
        display(Markdown("✍️ Bozze salvate:"))
        for idx, draft in enumerate(state["draft"], 1):
          display(Markdown(f"**Bozza {idx}**"))
          display(Markdown(f"{draft}"))
          display(Markdown(""))
          display(Markdown("-------------------------------------------------"))
    else:
        display(Markdown("📝 Nessuna post generata. Seleziona prima il topic dai suggerimenti!"))

#|----------------------------------------------------------------------------------------------------------------------------|

In [ ]:
workflow = StateGraph(BlogState)
workflow.add_node("Suggerimenti", suggest_topics)
workflow.add_node("Ricerca", research_topic)
workflow.add_node("Generazione", generate_post)
workflow.add_node("Stato", show_state)
workflow.add_node("Calendario", publication_calendar)


workflow.add_edge("Suggerimenti", "Ricerca")
workflow.add_edge("Ricerca", "Generazione")
workflow.add_edge("Generazione", "Stato")
workflow.add_edge("Stato", "Calendario")


workflow.set_entry_point("Suggerimenti")
workflow.set_finish_point("Calendario")
blog_agent = workflow.compile()

display(Image(blog_agent.get_graph().draw_mermaid_png()))

In [ ]:
def blogManager():
    state = {
        "suggested_topics": [],
        "draft": [],
        "draft_titles":[],
        "selected_topic": [],
        "last_research":[],
        "schedule_post": [],
        "memory": {}
    }

    while True:
        display(Markdown("|---------- Benvenuto nel BlogAgent ----------|"))
        display(Markdown("1. Genera suggerimenti\n2. Crea post\n3. Mostra stato\n4. Pianifica il calendario\n5. Esci"))
        display(Markdown("|---------------------------------------------|"))
        choice = input("Seleziona una opzione: ").strip()

        if choice == "5":
            break
        try:
            if choice == "1":
                user_input = input("\nInserisci l'argomento della guida pratica: ").strip()
                state = suggest_topics(state, user_input)
            elif choice == "2":
                generate_post(state)
            elif choice == "3":
                show_state(state)
            elif choice == "4":
                intervallo_giorni = int(input("\nInserisci l'intervallo di giorni tra i post: ").strip())
                state = publication_calendar(state, intervallo_giorni)
            else:
                display(Markdown("⚠️ Scelta non valida"))
        except Exception as e:
            display(Markdown(f"🔴 Errore: {str(e)}"))

if __name__ == "__main__":
    blogManager()